In [1]:
from DQbee import *
import sympy as sp
import time
import pandas as pd

------------------- Import Dqbee -------------------
---------- Dqbee Module loaded successfully ----------


### Multi-stability in the system

---

Consider the following system of ODEs:
$$
x' = -x (x - a) (x - 2a)
$$
where $a$ is positive.

In [2]:
a, x = sp.symbols('a x')
a = 1
multistable_eq_system = EquationSystem([sp.Eq(x, - x * (x - a) * (x - 2 * a))])
rhs = - x * (x - a) * (x - 2 * a)
rhs.expand()

-x**3 + 3*x**2 - 2*x

In [3]:
result = optimal_inner_quadratization(multistable_eq_system)

The Original System is: 


<IPython.core.display.Latex object>

The Optimal Dissipative Quadratization is: 


<IPython.core.display.Latex object>

The new introduced variables are: 


<IPython.core.display.Latex object>

The Optimal Quadratic Dissipative System is (with substitution): 


<IPython.core.display.Latex object>

In [4]:
result = dquadratization_multi_equilibrium(multistable_eq_system, [[0],[2]])

-------------------------- Compute a quadratization dissipative with multi given equilibrium --------------------------
The original system is: 


<IPython.core.display.Latex object>

-------------------------- The Dissipative quadratized system --------------------------


<IPython.core.display.Latex object>

-------------------------- The lambda value --------------------------
The lambda value is:  8
-------------------------- The Dissipative quadratized system with lambda value --------------------------


<IPython.core.display.Latex object>

-------------------------- The Jacobian matrix --------------------------


<IPython.core.display.Latex object>

In [5]:
import numpy as np

N = 10  # number of repetitions
a_list = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 50000, 100000]
methods = ['numpy', 'Routh-Hurwitz', 'sympy-naive']

results = []

for a in a_list:
    result = [a]
    for method in methods:
        times = []  # list to store the times of each run
        lambda_values = []  # list to store the lambda values of each run
        for _ in range(N):
            multistable_eq_system = EquationSystem([sp.Eq(x, - x * (x - a) * (x - 2 * a))])
            equilibrium = [[0], [2 * a]]
            start = time.time()
            lambda_value = dquadratization_multi_equilibrium(multistable_eq_system, equilibrium, method=method, display=False)[0]
            end = time.time()
            times.append((end - start) * 1000)  # transform time to milliseconds
            lambda_values.append(lambda_value)
        # calculate the average time and lambda value
        avg_time = round(np.mean(times), 2)
        result.append((lambda_values[0], avg_time, method))

    results.append(result)

data_frame = []
for result in results:
    data = []
    data.append(result[0])
    data.append(result[1][0])
    for i in range(1, len(result)):
        data.append(result[i][1])
    data_frame.append(data)

df = pd.DataFrame(data_frame, columns=['a','lambda', 'numpy', 'Routh-Hurwitz', 'sympy-naive'])
df

,a,lambda,numpy,Routh-Hurwitz,sympy-naive
0,1,8,39.19,37.55,35.96
1,2,32,36.14,38.57,38.34
2,5,128,36.13,39.38,41.97
3,10,512,37.72,39.01,43.39
4,20,2048,35.48,39.40,45.57
5,50,16384,35.20,39.78,51.47
6,100,65536,35.23,39.97,50.37
7,200,262144,35.41,40.15,52.81
8,500,1048576,38.39,40.84,55.40
9,1000,4194304,35.49,41.34,57.35
